In [1]:
!pip install --upgrade sagemaker

  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 10.0 MB/s eta 0:00:00
  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.1-py3-none-any.whl.metadata (4.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 63.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 21.9 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 89.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8

In [2]:
import boto3
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
s3 = boto3.client('s3')

# SageMaker 세션 생성
session = sagemaker.Session()

# 역할(권한) 설정
role = get_execution_role()

# S3 버킷 설정
bucket = 'lieon-data'
output_path = f's3://{bucket}/Output'

In [4]:
# PyTorch Estimator 설정
estimator = PyTorch(
    entry_point='train.py',  # 학습 스크립트
    source_dir='.',
    role=role,
    instance_count=1,  # 사용할 인스턴스 수
    instance_type='ml.g4dn.2xlarge',  # 사용할 인스턴스 타입 (GPU를 사용할 수 있음)
    framework_version='2.4.0',  # PyTorch 버전
    py_version='py311',
    output_path=output_path,  # 학습된 모델이 저장될 경로
    model_dir=False,
)

In [5]:
wav_dir_train = 's3://lieon-data/Dataset/Train/Audio'
label_dir_train = 's3://lieon-data/Dataset/Train/Label'

wav_dir_val = 's3://lieon-data/Dataset/Val/Audio'
label_dir_val = 's3://lieon-data/Dataset/Val/Label'

wav_dir_test = 's3://lieon-data/Dataset/Test/Audio'
label_dir_test = 's3://lieon-data/Dataset/Test/Label'

train_audio_input = TrainingInput(s3_data=wav_dir_train, content_type='wav')
train_label_input = TrainingInput(s3_data=label_dir_train, content_type='text/csv')

val_audio_input = TrainingInput(s3_data=wav_dir_val, content_type='wav')
val_label_input = TrainingInput(s3_data=label_dir_val, content_type='text/csv')

test_audio_input = TrainingInput(s3_data=wav_dir_test, content_type='wav')
test_label_input = TrainingInput(s3_data=label_dir_test, content_type='text/csv')

In [6]:
# S3에 저장된 데이터 경로 설정 (S3 오디오 및 라벨 파일 경로를 인자로 전달)
estimator.fit({
    'train_audio': train_audio_input,
    'train_label': train_label_input,
    'val_audio': val_audio_input,
    'val_label': val_label_input,
    'test_audio': test_audio_input,
    'test_label': test_label_input
})

S3UploadFailedError: Failed to upload /tmp/tmp9h_t3pft/source.tar.gz to lieon-data/pytorch-training-2024-10-10-07-28-47-564/source/sourcedir.tar.gz: An error occurred (AccessDenied) when calling the CreateMultipartUpload operation: User: arn:aws:sts::049840860985:assumed-role/AmazonSageMakerServiceCatalogProductsUseRole/SageMaker is not authorized to perform: s3:PutObject on resource: "arn:aws:s3:::lieon-data/pytorch-training-2024-10-10-07-28-47-564/source/sourcedir.tar.gz" because no identity-based policy allows the s3:PutObject action